In [3]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import re

import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
from bs4 import BeautifulSoup


In [10]:
engine = create_engine('sqlite:///data/data.db', echo=False)
df = pd.read_sql('data/data.db', con=engine, index_col='ID')

OperationalError: (sqlite3.OperationalError) near "data": syntax error
[SQL: data/data.db]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
df.head(3)

,title,category,content
ID,,,
0,피아니스트(로만 폴란스키 영화),2002년 영화홀로코스트 영화실화 바탕 영화폴란드 영화프랑스 영화영국 영화독일 영화...,로만 폴란스키 감독 장편 연출 작품 [ 펼치기 · 접기 ] 피아니스트의 주요 수상...
1,제1차 세계 대전,제1차 세계 대전,제1차 세계 대전 관련 틀 [ 펼치기 · 접기 ]제1차 세계 대전 참전국 [ 펼치기...
2,김영남,최고인민회의 상임위원장북한 외무상조선로동당 국제비서조선로동당 6기 정치국 상무위원조...,은(는) 여기로 연결됩니다. 동명이인에 대한 내용은 김영남(동명이인) 문서를의...


In [ ]:
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result

In [ ]:
print(text_cleaning(df['content'][0][:500]))

로만 폴란스키 감독 장편 연출 작품  펼치기  접기   피아니스트의 주요 수상 이력      펼치기  접기  선정 대 세기 영화   펼치기  접기   년 는 명의 국제 영화 평론가들을 대상으로 사상 최고의 세기 영화 편을 선정했다  위멀홀랜드 드라이브위화양연화위데어 윌 비 블러드위센과 치히로의 행방불명위보이후드위이터널 선샤인위트리 오브 라이프위하나 그리고 둘위씨민과 나데르의 별거위노인을 위한 나라는 없다위인사이드 르윈위조디악위칠드런 오브 맨위액트 오브 킬링위개월 주 그리고 일위홀리 모터스위판의 미로위하얀 리본위매드 맥스 분노의 도로위시네도키 뉴욕위그랜드 부다페스트 호텔위사랑도 통역이 되나요위히든위마스터위메멘토위시위소셜 네트워크위그녀에게위월위올드보이위마가렛위타인의 삶위다크 나이트위사울의 아들위


In [ ]:
for column in df.columns:
    column = str(column)
    df[column] = df[column].apply(lambda x : text_cleaning(x))

In [ ]:
df.head(3)

,title,category,content
ID,,,
0,피아니스트로만 폴란스키 영화,년 영화홀로코스트 영화실화 바탕 영화폴란드 영화프랑스 영화영국 영화독일 영화포커스 ...,로만 폴란스키 감독 장편 연출 작품 펼치기 접기 피아니스트의 주요 수상 이력...
1,제차 세계 대전,제차 세계 대전,제차 세계 대전 관련 틀 펼치기 접기 제차 세계 대전 참전국 펼치기 접기 관...
2,김영남,최고인민회의 상임위원장북한 외무상조선로동당 국제비서조선로동당 기 정치국 상무위원조선...,은는 여기로 연결됩니다 동명이인에 대한 내용은 김영남동명이인 문서를의 번 문단을...


In [ ]:
title_corpus = ''.join(df['title'].tolist())
category_corpus = ''.join(df['category'].tolist())
content_corpus = ''.join(df['content'].tolist())

In [5]:
print(title_corpus[:300])

NameError: name 'title_corpus' is not defined

In [6]:
!pip install konlpy
from konlpy.tag import Okt
from collections import Counter

In [7]:
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(content_corpus)
count = Counter(nouns)
print(list(count.items())[:10])

NameError: name 'content_corpus' is not defined

In [47]:
remove_char_counter = Counter({x : count[x] for x in count if len(x) > 1})
print(list(remove_char_counter.items())[:10])

[('로만', 29), ('폴란스키', 11), ('감독', 108), ('장편', 3), ('연출', 23), ('작품', 77), ('접기', 231), ('피아니스트', 33), ('주요', 94), ('수상', 35)]


In [51]:
# 불용어 제거 작업
korean_stopwords_path = 'korean_stopwords.txt'

In [52]:
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]
print(stopwords[:10])

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']


In [53]:
# 나무위키 페이지에 맞는 불용어를 추가합니다.
additional_wiki_stopwords = ['상위', '문서', '내용', '누설', '아래', '해당', '설명', '표기', '추가', '모든', '사용', '매우', '가장',
                       '줄거리', '요소', '상황', '편집', '틀', '경우', '때문', '모습', '정도', '이후', '사실', '생각', '인물', 
                       '이름', '년월']
for stopword in additional_wiki_stopwords:
    stopwords.append(stopword)

In [59]:
remove_char_counter = Counter({x : remove_char_counter[x] for x in count if x not in stopwords})
print(list(remove_char_counter.items())[:10][0])

('로만', 29)


In [2]:
print(remove_char_counter)

NameError: name 'remove_char_counter' is not defined

In [62]:
with open("data/keywords.txt", "w") as f:
    for item in list(remove_char_counter.items()):
        f.write(f'{item[0]},{item[1]}\n')